# Detailed Analysis of the predictions for a single run

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from infreqact.utils.predictions import load_predictions_jsonl

# # local path or from wandb (below)
# JSONL_FILENAME = "InternVL3_5-8B-HF_OOPS_cs_20260115-202011.jsonl"

# path = "../outputs/zeroshot-v2/predictions/" + JSONL_FILENAME
# metadata, predictions = load_predictions_jsonl(path)

In [ ]:
from infreqact.utils.wandb import load_run_from_wandb

PROJECT = "fall-detection-zeroshot-v3"
ENTITY = "moritzm00"

RUN_ID = "wycr9tin"  # 38B Internvl zeroshot
# RUN_ID = "s7b288b5" # 8B COT
config, predictions = load_run_from_wandb(RUN_ID, PROJECT, ENTITY)
metadata = {"config": config}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from loguru import logger
from omegaconf import OmegaConf

from infreqact.data.video_dataset_factory import get_video_datasets
from infreqact.utils.predictions import extract_labels_for_metrics

sns.set_theme(
    style="white",
    context="paper",
    rc={"text.usetex": True, "font.family": "serif", "font.serif": ["Computer Modern Roman"]},
)

y_true, y_pred = extract_labels_for_metrics(predictions)

cfg = metadata["config"]
cfg = OmegaConf.create(cfg)

# optionally change dataset parameters for visualizations
cfg.dataset.vid_frame_count = 16
cfg.dataset.model_fps = 8

seed = cfg.data.seed
dataset = get_video_datasets(cfg, mode="test", split="cs", size=768, seed=seed)

preds = pd.DataFrame(predictions).rename(
    columns={"predicted_label": "y_pred", "label_str": "y_true"}
)

In [ ]:
import torch
import torchvision.utils as vutils


def video_to_image_grid(
    video: torch.Tensor, nrow: int | None = None, padding: int = 2, normalize: bool = True
) -> torch.Tensor:
    """
    Convert a video tensor to an image grid.

    Args:
        video: Tensor of shape (T, C, H, W) where T is number of frames
        nrow: Number of images per row. If None, uses ceil(sqrt(T))
        padding: Padding between images
        normalize: Whether to normalize the output to [0, 1]

    Returns:
        Image grid tensor of shape (C, H', W') suitable for display
    """
    T, C, H, W = video.shape

    if nrow is None:
        nrow = int(torch.ceil(torch.sqrt(torch.tensor(T, dtype=torch.float))).item())

    # Use torchvision's make_grid utility
    grid = vutils.make_grid(video, nrow=nrow, padding=padding, normalize=normalize)

    return grid

In [ ]:
def visualize_prediction(
    idx: int, figsize=(16, 8), title=False, nrow: int | None = 8, savepath: str | None = None
):
    logger.info(f"Visualizing prediction for index: {idx}")
    video = dataset[idx]["video"]
    predicted_label = y_pred[idx]
    true_label = y_true[idx]
    caption = f"Predicted: {predicted_label}, True: {true_label}"
    logger.info(caption)

    # print reasoning if available
    reasoning = preds.loc[idx, "reasoning"]
    if reasoning != "":
        print("Reasoning:\n", reasoning)

    grid = video_to_image_grid(video, nrow=nrow)

    fig, ax = plt.subplots(figsize=figsize)
    plt.imshow(grid.permute(1, 2, 0))
    plt.axis("off")
    if title:
        plt.title(caption)
    if savepath is not None:
        plt.savefig(savepath)
    return ax

In [ ]:
path = predictions[122]["video_path"]
full_path = dataset.datasets[0].format_path(path)
full_path

In [ ]:
import decord

# Load the video at 8 FPS
decord.bridge.set_bridge("torch")
vr = decord.VideoReader(full_path)
original_fps = vr.get_avg_fps()
total_frames = len(vr)
duration = total_frames / original_fps

# Calculate frame indices for 8 FPS
target_fps = 8
frame_interval = original_fps / target_fps
frame_indices = np.arange(0, total_frames, frame_interval).astype(int)
frame_indices = frame_indices[frame_indices < total_frames]

# Get timestamps for each sampled frame
timestamps = frame_indices / original_fps

# Load frames
frames = vr.get_batch(frame_indices)  # (T, H, W, C)

print(f"Original FPS: {original_fps:.2f}, Duration: {duration:.2f}s")
print(f"Sampled {len(frame_indices)} frames at {target_fps} FPS")
print(f"Frame shape: {frames.shape}")
print(f"Timestamps (first 10): {timestamps[:10]}")

In [ ]:
from IPython.display import HTML
from matplotlib import animation

# Get segment info from predictions
pred = predictions[122]
start_time = pred["start_time"]
end_time = pred["end_time"]

# Calculate frame indices for the segment at 8 FPS
start_frame = int(start_time * original_fps)
end_frame = int(end_time * original_fps)

# Sample at target FPS within the segment
segment_frame_indices = np.arange(start_frame, end_frame, frame_interval).astype(int)
segment_frame_indices = segment_frame_indices[segment_frame_indices < total_frames]

# Get frames for this segment
segment_frames = vr.get_batch(segment_frame_indices)  # (T, H, W, C)

print(f"Segment: {start_time:.2f}s - {end_time:.2f}s (duration: {end_time - start_time:.2f}s)")
print(f"Frames: {len(segment_frame_indices)} at {target_fps} FPS")
print(f"True label: {pred['label_str']}, Predicted: {pred['predicted_label']}")

# Visualize as video using matplotlib animation

fig, ax = plt.subplots(figsize=(8, 6))
ax.axis("off")

# Convert to numpy for display
frames_np = segment_frames.numpy()

im = ax.imshow(frames_np[0])
ax.set_title(f"True: {pred['label_str']} | Pred: {pred['predicted_label']}")


def update(frame_idx):
    im.set_array(frames_np[frame_idx])
    return [im]


ani = animation.FuncAnimation(
    fig, update, frames=len(frames_np), interval=1000 / target_fps, blit=True
)
plt.close(fig)

HTML(ani.to_jshtml())

In [ ]:
# visualize n random idx
rng = np.random.default_rng(0)
idx = rng.integers(low=0, high=len(dataset), size=8)
for i in idx:
    pred = predictions[i]
    true_label = pred["label_str"]
    visualize_prediction(idx=i, nrow=8, title=False)
    plt.savefig(f"../outputs/plots/pred_{i}_{true_label}.pdf")
    plt.show()

In [ ]:
visualize_prediction(
    45, nrow=8, savepath="../outputs/plots/pred_45_fall_wide.pdf", figsize=(36, 12)
)

In [ ]:
# Create a combined DataFrame for comparison
df_compare = pd.DataFrame(
    {"label": y_true + y_pred, "type": ["True"] * len(y_true) + ["Predicted"] * len(y_pred)}
)

# Set up the figure
plt.figure(figsize=(14, 6))

# Count plot comparing distributions
ax = sns.countplot(data=df_compare, x="label", hue="type")  # , palette=["#2ecc71", "#e74c3c"])
# plt.title("Distribution of Predicted vs True Labels", fontsize=14)
plt.xlabel("Action Label", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.xticks(rotation=45, ha="center")
plt.legend(title="Label Type")
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Get unique labels in order of appearance
labels = sorted(set(y_true) | set(y_pred))

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=labels)

# Normalize by row (true labels) to show recall per class
cm_normalized = cm.astype("float") / cm.sum(axis=1, keepdims=True)

# Create annotation matrix with percentage and count (only for values >= 3%)
annot_matrix = np.empty_like(cm, dtype=object)
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        if cm_normalized[i, j] >= 0.03:
            annot_matrix[i, j] = f"{cm_normalized[i, j]:.2f}\n({cm[i, j]})"
        else:
            annot_matrix[i, j] = ""

# Create mask for values below 3%
mask = cm_normalized < 0.03

# Plot
plt.figure(figsize=(12, 10))
with sns.axes_style("white"):
    sns.heatmap(
        cm_normalized,
        annot=annot_matrix,
        fmt="",
        cmap="Blues",
        xticklabels=labels,
        yticklabels=labels,
        square=True,
        cbar_kws={"label": "Proportion"},
        mask=mask,
    )
# plt.title("Confusion Matrix (Row-Normalized)", fontsize=14)
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.xticks(rotation=45, ha="center")
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("../outputs/plots/confusion_matrix.pdf")
plt.show()

## Segment Duration vs Accuracy

In [ ]:
preds.segment_duration.hist(bins=30, figsize=(10, 6))

In [ ]:
# Add 'correct' column to track predictions
preds["correct"] = preds["y_true"] == preds["y_pred"]

preds["duration_bin"] = pd.cut(
    preds["segment_duration"],
    bins=[0, 2, 3, 4, 5, 10, float("inf")],
    labels=["0-2s", "2-3s", "3-4s", "4-5s", "5-10s", "10s+"],
)
# Calculate accuracy per bin
accuracy_by_duration = (
    preds.groupby("duration_bin", observed=True)["correct"].agg(["mean", "count"]).reset_index()
)
accuracy_by_duration.columns = ["Duration", "Accuracy", "Sample Count"]

# Plot
fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar plot for accuracy
bars = sns.barplot(data=accuracy_by_duration, x="Duration", y="Accuracy", ax=ax1, color="steelblue")
ax1.set_ylabel("Accuracy", fontsize=12)
ax1.set_xlabel("Segment Duration", fontsize=12)
ax1.set_ylim(0, 1)

# Add sample counts as text on bars
for i, (acc, count) in enumerate(
    zip(accuracy_by_duration["Accuracy"], accuracy_by_duration["Sample Count"])
):
    ax1.text(i, acc + 0.02, f"n={count}", ha="center", fontsize=10)

plt.title("Classification Accuracy by Segment Duration", fontsize=14)
plt.tight_layout()
plt.show()

## Per-Class Performance Analysis

In [ ]:
# Calculate per-class metrics
class_metrics = []
for label in labels:
    true_mask = preds["y_true"] == label
    pred_mask = preds["y_pred"] == label

    tp = (true_mask & pred_mask).sum()
    fp = (~true_mask & pred_mask).sum()
    fn = (true_mask & ~pred_mask).sum()

    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    class_metrics.append(
        {
            "class": label,
            "support": true_mask.sum(),
            "recall": recall,
            "precision": precision,
            "f1": f1,
        }
    )

class_df = pd.DataFrame(class_metrics).sort_values("f1")

# Plot per-class F1 scores
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.barh(class_df["class"], class_df["f1"])
ax.set_xlabel("F1 Score")
# ax.set_title("Per-Class F1 Score", fontsize=14)
ax.set_xlim(0, 0.8)

# Add vertical grid lines
ax.xaxis.grid(True, linestyle="--", alpha=0.7)
ax.set_axisbelow(True)

# Add support counts
for i, (f1, support) in enumerate(zip(class_df["f1"], class_df["support"])):
    ax.text(f1 + 0.02, i, f"n={support}", va="center", fontsize=9)

plt.tight_layout()
plt.savefig("../outputs/plots/per_class_f1.pdf")
plt.show()

# Show detailed metrics table
print("\n📊 Per-Class Metrics (sorted by F1):")
print(class_df.to_string(index=False, float_format=lambda x: f"{x:.2f}"))

In [ ]:
# Confusion analysis: What does each class get confused with?
print("🔍 Misclassification Analysis per Class:\n")

for label in labels:
    true_mask = preds["y_true"] == label
    errors = preds[true_mask & ~preds["correct"]]

    if len(errors) > 0:
        error_dist = errors["y_pred"].value_counts().head(3)
        total_true = true_mask.sum()
        print(
            f"【{label}】 (n={total_true}, errors={len(errors)}, recall={1 - len(errors) / total_true:.1%})"
        )
        for wrong_pred, count in error_dist.items():
            pct = count / total_true * 100
            print(f"   → predicted as '{wrong_pred}': {count} ({pct:.1f}%)")
        print()

In [ ]:
# Heatmap showing top confusions (filtered to significant errors)
# Create a DataFrame of error rates: what % of class X is predicted as class Y
error_matrix = pd.DataFrame(0.0, index=labels, columns=labels)

for true_label in labels:
    true_mask = preds["y_true"] == true_label
    total = true_mask.sum()
    if total > 0:
        pred_counts = preds[true_mask]["y_pred"].value_counts()
        for pred_label, count in pred_counts.items():
            if pred_label != true_label:  # Only off-diagonal (errors)
                error_matrix.loc[true_label, pred_label] = count / total * 100

# Plot only significant confusions
plt.figure(figsize=(14, 10))

# Create custom annotations with % symbol
annot_matrix = error_matrix.map(lambda x: f"{x:.0f}%" if x >= 5 else "")

sns.heatmap(
    error_matrix,
    annot=annot_matrix,
    fmt="s",  # 's' for string format
    cmap="Reds",
    cbar_kws={"label": "Error Rate (%)"},
    mask=(error_matrix < 5),  # Hide small errors for clarity
    linewidths=0.5,
)
plt.title(r"Confusion Error Rates (showing only $\geq$5\%)", fontsize=14)
plt.xlabel("Predicted As", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

## Visualize Predictions

In [ ]:
def visualize_by_class(
    actual_class: str = None, predicted_class: str = None, sample_idx: int = 0, **kwargs
):
    """
    Visualize predictions filtered by actual and/or predicted class.

    Args:
        actual_class: Filter by ground truth label
        predicted_class: Filter by predicted label
        sample_idx: Which sample to show (0 = first match, 1 = second match, etc.)

    Examples:
        visualize_by_class(actual_class="fall")  # Show a sample where true label is "fall"
        visualize_by_class(predicted_class="fall")  # Show a sample predicted as "fall"
        visualize_by_class(actual_class="jump", predicted_class="fall")  # Show "jump" misclassified as "fall"
    """
    if actual_class is None and predicted_class is None:
        raise ValueError("Must specify at least one of actual_class or predicted_class")

    # Build filter mask
    mask = pd.Series([True] * len(preds))

    if actual_class is not None:
        mask &= preds["y_true"] == actual_class
    if predicted_class is not None:
        mask &= preds["y_pred"] == predicted_class

    matching_indices = preds[mask].index.tolist()

    if len(matching_indices) == 0:
        filter_desc = []
        if actual_class:
            filter_desc.append(f"actual='{actual_class}'")
        if predicted_class:
            filter_desc.append(f"predicted='{predicted_class}'")
        print(f"No samples found with {' and '.join(filter_desc)}")
        return

    if sample_idx >= len(matching_indices):
        print(
            f"Only {len(matching_indices)} samples match. Use sample_idx < {len(matching_indices)}"
        )
        sample_idx = len(matching_indices) - 1

    idx = matching_indices[sample_idx]

    # Show info
    filter_desc = []
    if actual_class:
        filter_desc.append(f"actual='{actual_class}'")
    if predicted_class:
        filter_desc.append(f"predicted='{predicted_class}'")
    print(
        f"Showing sample {sample_idx + 1}/{len(matching_indices)} matching {' and '.join(filter_desc)}"
    )

    visualize_prediction(idx, **kwargs)


# Example usage:
# visualize_by_class(actual_class="jump", predicted_class="fall")  # Show jump→fall confusion
# visualize_by_class(actual_class="fall", predicted_class="fall")  # Show correct fall prediction
# visualize_by_class(predicted_class="crawl")  # Show anything predicted as crawl

In [ ]:
visualize_by_class(predicted_class="crawl", title=False)

In [ ]:
visualize_by_class(
    actual_class="jump", predicted_class="fall", sample_idx=0, nrow=8, figsize=(16, 8)
)
plt.savefig("../outputs/plots/pred_jump_fall.pdf")

In [ ]:
visualize_by_class(actual_class="fallen", predicted_class="fall")